In [ ]:
import numpy as np

In [ ]:
#Fonctions

def rad2dms(a):
    '''rad en deg,m,s'''
    r=rad2deg(a)
    d=int(r)
    r-=d
    m=int(r*60)
    r-=m/60
    s=r*3600
    return d, m, s

def dms2rd(d,m,s):
    '''deg,m,s en rad'''
    return deg2rad(d)+deg2rad(m/60)+deg2rad(s/3600)


def hsexag2hdec(h):
    '''heure sexagecimal sous forme decimal en heure decimal'''
    heure=int(h)
    dec=h-heure
    mn=(dec*100/60)
    return heure+mn

def hdec2hsexag(h):   
    '''heure decimal en heure sexagecimal + fuseau + dst'''
    heure=int(h)
    dec=h-heure
    mn=int(dec*60)
    s=int((dec-mn/60)*3600)
    return (heure+fuseau[0]+dst,mn,s)

def deg2rad(a):
    '''rad en deg decimal'''
    return a/180*np.pi

def rad2deg(a):
    ''' rad en degr decimal'''
    return a/np.pi*180

def deg2hms(deg):
    '''deg en h,m,s'''
    h=int(deg/15)
    #print(h)
    mn=(deg-h*15)
    m=int(mn/.25)
    #print(m)
    sec=mn-m*.25
    s=int((sec/.004167))
    #print(s)
    return (h,m,s)

def sec2hms(s):
    '''sec en h,m,s'''
    h=int(s/3600)
    #print(h)
    mn=(s-h*3600)
    m=int(mn/60)
    #print(m)
    s=int(mn-m*60)
    return (h,m,s)

def degseg2ms(deg):
    '''format decimal deg.mn  -> heure, mn, sec'''
    return (0,int(deg)*4,int((deg-int(deg))*40))  # 1 deg=4mn, 1'=4sec

def fprint(list):
    '''print hh:mm:ss'''
    print('{}:{}:{}'.format(int(list[0]),int(list[1]),int(list[2])))
    
def addlist(l1,l2):
    '''additionne deux heures en liste (h,m,s)'''
    a1=np.array(l1)
    a2=np.array(l2)
    a=a1+a2
    a[1]+=a[2]//60
    a[2]%=60
    a[0]+=a[1]//60
    a[1]%=60
    return list(a)
 
def sublist(l1,l2):
    '''soustraction de deux heures en liste (h,m,s)'''
    a1=np.array(l1)
    a2=np.array(l2)
    if a1[1]<a2[1]:
        a1[0]-=1
        a1[1]+=60
    if a1[2]<a2[2]:
        a1[1]-=1
        a1[2]+=60
    return list(a1-a2)


def convEcl2Equ(l,b):    
    '''coord. ecliptiques en coord. equatoriales'''
    D=np.cos(e)*np.sin(b)+np.sin(e)*np.cos(b)*np.sin(l)
    d=np.arcsin(D)
    A=(-np.sin(e)*np.sin(b)+np.cos(e)*np.cos(b)*np.sin(l))/np.cos(d)
    a=np.arcsin(A)
    if np.cos(b)*np.sin(l)<0:a+=np.pi
    return rad2deg(a),rad2deg(d)

def convEqu2Ecl(a,d):  
    '''coord. equatoriales en coord. ecliptiques'''
    B=np.cos(e)*np.sin(d)-np.sin(e)*np.cos(d)*np.sin(a)
    b=np.arcsin(B)
    L=(np.sin(e)*np.sin(d)+np.cos(e)*np.cos(d)*np.sin(a))/np.cos(b)
    l=np.arcsin(L)
    if np.cos(d)*np.sin(a)<0:l+=np.pi
    return rad2deg(l),rad2deg(b)

def semiarc(lat,dec):
    '''calcul du semiarc diurne'''
    DA=np.tan(lat)*np.tan(dec)
    #print(DA)
    da=np.arcsin(DA)
    return rd(da),90+rd(da)

def daysfromref(d,m,y):  
    '''
    numbers days from 0/1/1901
    constant for 0/11901=694325
    return : days after 0/1/1901, day of week (0 is Sunday,...)
    '''
    if m<3:
        r=365*y+int((y-1)/4)+31*(m-1)-int((y-1)/100)+int((y-1)/400)+d
        return r-694325,r%7
    else:
        r=365*y+int(y/4)+31*(m-1)-int((m-1)*.4+2.7)-int((y)/100)+int((y)/400)+d
        return r-694325,r%7   

#sideral time TS
def ts_s(n):
    '''
    Numbers of seconds from 0/1/1901 at 0h modulo 86400
    input : n days from 0/1/1901
    '''
    return (23750.3+236.555362*n)%86400    

def ts_d(n):
    '''TS in degrees modulo 360
    give AD of gamma point
    '''
    return (98.965+.985647342*n)%360

def computeN(d,m,y):
    '''give numbers of days from 0/1/1901'''
    mon=[0,31,59.25,90.25,120.25,151.25,181.25,212.25,243.25,273.25,304.25,334.25]
    A=y-1901
    A*=365.25
    A+=d
    A+=mon[m-1]
    return int(A)

def TS_s(d,m,y):
    '''combo of functions'''
    return ts_s(computeN(d,m,y))

def TS_d(d,m,y):
    '''combo of functions'''
    return ts_d(computeN(d,m,y))

def isDst(dst,d):
    '''calcul du dst suivant la declinaison du soleil'''
    if (dst==0 and d>4): 
        dst=1    
    elif (dst==1 and d<-12):
        dst=0
    return dst

def heurelever(lat,decl):
    d=deg2rad(decl)
    l=deg2rad(lat)
    hl=np.arccos(np.tan(l)*np.tan(d))/deg2rad(15)
    hc=24-hl
    return hl,hc

def hauteur(lat,decl):
    return 90-(lat-decl) if (lat-decl)>0 else 90+lat-decl

In [ ]:
#Constantes

DOW=['Dim','Lun','Mar','Mer','Jeu','Ven','Sam']
#gregorian  15,10,1582
#(y>=1582 and m>=10 and d>4)
#constante julien=1721059,5
#0 janvier 1901    N=694325
fuseau=(1,0,0)
dst=1
d=23.45
e=deg2rad(23.45)
print(e)

In [ ]:
dst=isDst(dst,d)
print(dst)
test=sec2hms(2644)
print('sec2hms test',test)

In [ ]:
#function aliases
hd=hsexag2hdec
hs=hdec2hsexag
dr=deg2rad
rd=rad2deg
ec=convEqu2Ecl
eq=convEcl2Equ
d2h=deg2hms
mns=degseg2ms
s2h=sec2hms

In [ ]:
#test1    
print('1=============================================')     
test=sec2hms(3601)
print('sec2hms test',test)
print('test daysfromref')
N=computeN(3,5,1973)    
print('N=',N)


# ddd = N avec 2 formules computeN ou daysfromref
ddd,dow=daysfromref(3,5,1973)   
print(ddd,DOW[dow-1])  
ddd,dow=daysfromref(1,1,1901)   
print(ddd,DOW[dow-1]) 

ddd,dow=daysfromref(27,6,2019)   
print(ddd,dow,DOW[dow-1]) 

In [ ]:
#test2
print('2=============================================')
#conversion function test
print('Ecliptic to Equatorial test')
a, d = convEcl2Equ(deg2rad(197.5),deg2rad(1.4))
print('a=',a, 'd=',d)
print('Equatorial to Ecliptic  test')
l, b = convEqu2Ecl(deg2rad(196.67),deg2rad(-5.578))
print('l=',l,'b=', b)

In [ ]:
#test3
print('3=============================================')
print('Sideral time Test')
print('19 Juin 2019')
N=daysfromref(19,6,2019)[0]
print(ts_s(N),ts_d(N))   #return seconds or degrees
print(s2h(ts_s(N)),d2h(ts_d(N)))

TS0=d2h(ts_d(N))  #temps sideral de greenwich-0h
TS=d2h(TS_d(19,6,2019))
print(TS0,TS)

print('Troyes')
lon=mns(4.5)     #longitude de Troyes, lieu d'observation
print (lon)
heure_local=(12,30,0)

h1=TS0   #TS0 temps sideral pour greenwich-0h
print('TS0=',h1)
h2=sublist(heure_local,fuseau)  #heure TU de l'observation = heure locale - fuseau
print('TU=',h2)
h3=addlist(h1,h2)  #TS1 temps sideral greenwich a l'heure d'observation
print('TS1=',h3)
tsl=addlist(h3,lon)  #TSL : temps sideral au longitude du point d'observation
print('Temps sideral local=',end=' ')
fprint(tsl)

In [ ]:
#test4
print('4=============================================')
print('Date d observation')
#date input
d,m,y=27,6,2019
print(d,'/',m,'/',y)

In [ ]:
print('Compute N')
N=computeN(d,m,y)    
print('N =',N)
ddd,dow=daysfromref(d,m,y)   
print('Verif:',ddd,dow,DOW[dow-1]) 

In [ ]:
print('Lieu d observation : Paris')
#Observation Paris
lat=(48.8534)      #latitude de Paris
#lon=0              #longitude de Greenwich
lon=(2.3488)        #longitude de Paris
print(lat,lon)

In [ ]:
#Saturn -----------------------------
#name'Saturne'
#N=26421       #value
#N=computeN(3,5,1973)    #computing for the date Saturn
#Objet celeste a observer
#Saturne 
#s_lon=dr(78.95)     #longitude de Saturne
#s_lat=dr(-1.17)     #latitude Saturne
#HIP118307

name='Gamma Point'                # HIP 118307 Poissons
print('Objet a observer :',name)


In [ ]:
#coord. ecliptiques
print('Coordonnees ecliptiques:',name)
s_lon=dr(0)     #longitude du point Gamma   
s_lat=dr(0)     #latitude du point gamma  
print('lon=',s_lon)
print('lat=',s_lat)

In [ ]:
#coordonnees ecliptiques en equatoriales
print('Coordonnees equatoriales:',name)
a,d=eq(s_lon,s_lat)
print('AD (deg)=',a)
a_h=d2h(a)
print('AD (h)=',a_h)
print('DEC=',d)
#ici on a les valeurs de a,d et a_h
print()

In [ ]:
print('Le calcul commence ici ')
name='Soleil'
date=(27,6,2019)
print('exemple : ',name,date)

In [ ]:
# on peut rentrer directement les coord. equatoriales
#a_h=ascension droite en (h,m,s), d=declinaison en deg
#en degres decimal

print('Input AD, d en liste (h,m,s):  AD et d donnes dans Redshift')
a_h=(6,24,0.3)              #AD en h,m,s 
decl=(23,19,11)              #decl en h,m,s
d=rd(dms2rd(decl[0],decl[1],decl[2]))           #d en deg decimal
print('Ascension droite=',a_h,'- Declinaison=',decl)
hc=hauteur(lat,d)
print('Hauteur Culmination:',hc)

In [ ]:
#Calcul du demi arc diurne
print('Calcul du demi-arc diurne pour Paris :')
da, sa = semiarc(dr(lat),dr(d))
#print('Azimuth : ',da)
print('Demi-arc diurne(deg)=',sa)
#Culmination hour 
#sa_h : semiarc in hour
sa_h=d2h(abs(sa))
print('Demi-arc diurne(h)=',sa_h)

In [ ]:
print('Calcul du temps sideral',date,N)
TS0=d2h(ts_d(N))
print('TS0',TS0)
#print('AD',a_h)

In [ ]:
print('Formules de calcul:')
print('Culmination=TS0-24+a_h')
print('Coucher=Culmination+sa_h')
print('Lever=Culmination-sa_h')

In [ ]:
t1=sublist((24,0,0),TS0)   #retire 24h
print('TS0-24=',t1)

tc=addlist(t1,a_h)    #calcul heure culmination 
ts=addlist(tc,sa_h)   #calcul du coucher
tr=sublist(tc,sa_h)   #calcul du lever

print('Heure de Greenwich :')
print(name+' lever :',end=' ')
fprint(tr)
print(name+' culmine a : ',end=' ')
fprint(tc)
print(name+' coucher :',end=' ')
fprint(ts)

In [ ]:
print ('Heure de Paris :')
dst=isDst(dst,d)
print('dst=',dst)
#ajout de dst et fuseau horaire
print(fuseau)
DST=(fuseau[0]+dst,0,0)
print('+DST = ',end='')
fprint(DST)

In [ ]:
# paris is east of greenwich 
# then culmination is sooner
# we substract the longitude in hours
print('Longitude Paris (deg, heure): ',lon,d2h(lon)) 
print('=> Paris est en avance : ',end='')
fprint(d2h(lon))   
tcParis=sublist(tc,d2h(lon))       #avance = soustraction
print('Culmine a Paris (heure solaire) : ',end='')  
fprint(tcParis) 

In [ ]:
print('Heure legale Paris: TU +',DST[0],date)
h_legal=addlist(tcParis,DST)
h_lever=sublist(h_legal,sa_h)
h_coucher=addlist(h_legal,sa_h)

print('Culmination : ',end='') 
fprint(h_legal)
print('Hauteur Culmination:',hc)
print('Lever : ', end='')
fprint(h_lever) 
print('Coucher : ', end='')
fprint(h_coucher) 
#autre calcul lever,coucher heure greenwich+DST
print('Calcul par heurelever()')
lever,coucher=heurelever(lat,d)
print(hdec2hsexag(lever),hdec2hsexag(coucher))
#print(lever+DST[0],coucher+DST[0])

In [ ]:
print('N:0, NE:45, E:90, SE:135, S:180,  SO:225, O:270, NO:315')
print()
print('Azimuth Lever : ', end='')
print(180-sa) 
print('Azimuth Coucher : ', end='')
print(180+sa) 